In [1]:
import os
import re
import numpy as np
import pandas as pd
from collections import defaultdict

base_dir = "."  # Adjust path if needed

# Config map for CKKS configurations
config_map = {
    "light": "CKKS Light",
    "med": "CKKS Medium",
    "heavy": "CKKS Heavy"
}

server_metrics = ["avg_cpu", "peak_cpu", "avg_mem", "peak_mem"]
config_round_data = {label: defaultdict(list) for label in config_map.values()}

# === Parse each config ===
for config_prefix, label in config_map.items():
    for folder in os.listdir(base_dir):
        if folder.startswith(f"exp_{config_prefix}_r"):
            try:
                server_log = [f for f in os.listdir(os.path.join(base_dir, folder)) if f.startswith("log_server_")][0]
                file_path = os.path.join(base_dir, folder, server_log, "resource_usage.txt")

                with open(file_path, "r") as f:
                    current_round = None
                    round_data = {}

                    for line in f:
                        round_match = re.match(r"Round (\d+):", line)
                        if round_match:
                            if current_round is not None:
                                config_round_data[label][current_round].append(round_data)
                            current_round = int(round_match.group(1))
                            round_data = {}
                        else:
                            metric_match = re.match(r"\s*(\w+):\s*([\d.]+)", line)
                            if metric_match:
                                key, val = metric_match.groups()
                                val = float(val)
                                if key in server_metrics:
                                    round_data[key] = val

                    if current_round is not None:
                        config_round_data[label][current_round].append(round_data)
            except Exception as e:
                print(f"❌ Error parsing {folder}: {e}")

# === Create summary tables ===
summary_tables = {}

for config, rounds_dict in config_round_data.items():
    round_numbers = sorted(rounds_dict.keys())
    summary_data = []

    for r in round_numbers:
        entry = {"Round": r}
        for metric in server_metrics:
            values = [run[metric] for run in rounds_dict[r] if metric in run]
            if values:
                entry[f"{metric} Mean"] = np.mean(values)
                entry[f"{metric} Std"] = np.std(values)
        summary_data.append(entry)

    df_summary = pd.DataFrame(summary_data).set_index("Round").round(2)
    summary_tables[config] = df_summary


In [2]:
# === Compute average across all rounds for summary ===
server_summary_rows = []

for config, df in summary_tables.items():
    row = {"Config": config}
    for metric in ["avg_cpu", "peak_cpu", "avg_mem", "peak_mem"]:
        col_name = f"{metric} Mean"
        row[metric] = round(df[col_name].mean(), 2) if col_name in df else "-"
    server_summary_rows.append(row)

# Create summary DataFrame
server_summary_df = pd.DataFrame(server_summary_rows).rename(columns={
    "avg_cpu": "Avg CPU (%)",
    "peak_cpu": "Peak CPU (%)",
    "avg_mem": "Avg Memory (MB)",
    "peak_mem": "Peak Memory (MB)"
}).set_index("Config")

display(server_summary_df)


,Avg CPU (%),Peak CPU (%),Avg Memory (MB),Peak Memory (MB)
Config,,,,
CKKS Light,2.79,33.42,1450.83,1532.14
CKKS Medium,2.32,29.14,1474.02,1584.01
CKKS Heavy,3.33,36.72,1373.56,1494.20


In [3]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict

base_dir = "."  # or path to your experiment root

# Map config folder prefix to label
config_map = {
    "light": "CKKS Light",
    "med": "CKKS Medium",
    "heavy": "CKKS Heavy"
}

# Metrics to extract
cpu_metrics = ["Avg CPU (%)", "Peak CPU (%)"]
mem_metrics = ["Avg Memory (MB)", "Peak Memory (MB)"]

# Storage
cpu_data = {label: {metric: defaultdict(list) for metric in cpu_metrics} for label in config_map.values()}
mem_data = {label: {metric: defaultdict(list) for metric in mem_metrics} for label in config_map.values()}

# Walk all experiment folders
for folder in os.listdir(base_dir):
    for cfg_prefix, label in config_map.items():
        if folder.startswith(f"exp_{cfg_prefix}_r"):
            try:
                log_folder = [
                    f for f in os.listdir(os.path.join(base_dir, folder))
                    if f.startswith("log_clients_")
                ][0]
                csv_path = os.path.join(base_dir, folder, log_folder, "client_0", "metrics_log.csv")
                df = pd.read_csv(csv_path)

                for _, row in df.iterrows():
                    round_num = int(row["Round"])
                    for metric in cpu_metrics:
                        value = row.get(metric, None)
                        if pd.notnull(value):
                            cpu_data[label][metric][round_num].append(value)
                    for metric in mem_metrics:
                        value = row.get(metric, None)
                        if pd.notnull(value):
                            mem_data[label][metric][round_num].append(value)
            except Exception as e:
                print(f"❌ Failed to load from {folder}: {e}")


In [4]:
# Combine CPU and Memory into one summary table
summary_rows = []

for config in config_map.values():
    row = {"Config": config}

    for metric in cpu_metrics:
        all_vals = []
        for round_vals in cpu_data[config][metric].values():
            all_vals.extend(round_vals)
        row[metric] = round(np.mean(all_vals), 2) if all_vals else "-"

    for metric in mem_metrics:
        all_vals = []
        for round_vals in mem_data[config][metric].values():
            all_vals.extend(round_vals)
        row[metric] = round(np.mean(all_vals), 2) if all_vals else "-"

    summary_rows.append(row)

client_summary_df = pd.DataFrame(summary_rows).set_index("Config")
display(client_summary_df)


,Avg CPU (%),Peak CPU (%),Avg Memory (MB),Peak Memory (MB)
Config,,,,
CKKS Light,40.88,48.41,1476.11,1483.40
CKKS Medium,41.76,48.88,1433.68,1439.99
CKKS Heavy,38.50,48.20,1508.79,1518.01
